In [2]:
// Car.java
public class Car {
    private String model;
    private Owner owner; // Reference to the owner

    // Constructor
    public Car(String model) {
        this.model = model;
    }

    // Getters and Setters
    public Person getOwner() {
        return owner;
    }

    public void setOwner(Person owner) {
        this.owner = owner;
    }

    public String getModel() {
        return model;
    }

    public void setModel(String model) {
        this.model = model;
    }
}

CompilationException: 

In [ ]:
// Owner.java
import java.util.ArrayList;
import java.util.Collections;
import java.util.List;

public class Owner {
    private String name;
    private List<Car> cars;

    public Owner(String name) {
        this.name = name;
        this.cars = new ArrayList<>(); // Composition
    }

    // Adds a car and sets the owner of the car
    public void addCar(Car car) {
        if (car == null) return;
        if (!cars.contains(car)) {
            cars.add(car);
            car.setOwner(this); // Set the bi-directional relationship
        }
    }

    // Removes a car and unsets the owner of the car
    public void removeCar(Car car) {
        if (car == null) return;
        if (cars.contains(car)) {
            cars.remove(car);
            car.setOwner(null); // Remove the Car -> Owner relationship
        }
    }

    // Getters and Setters
    public String getName() {
        return name;
    }

    public void setName(String name) {
        this.name = name;
    }

    /** Returns an unmodifiable view to avoid external mutation. */
    public List<Car> getCars() {
        return Collections.unmodifiableList(cars);
    }

    /** Replace all cars; re-wire ownerships accordingly. */
    public void setCars(List<Car> cars) {
        // clear old links
        for (Car c : new ArrayList<>(this.cars)) {
            c.setOwner(null);
        }
        this.cars.clear();

        if (cars != null) {
            for (Car c : cars) {
                addCar(c); // ensures bi-directional link
            }
        }
    }
}

In [3]:
public class Object {
    private String name;

    public Object(String name) {
        this.name = name;
    }

    public String getName() { return name; }

    public void setName(String name) { this.name = name; }
}

In [4]:
// Car.java
public class Car {
    private String model;
    private Person owner; // Reference to the owner

    // Constructor
    public Car(String model) {
        this.model = model;
    }

    // Getters and Setters
    public Person getOwner() {
        return owner;
    }

    public void setOwner(Person owner) {
        this.owner = owner;
    }

    public String getModel() {
        return model;
    }

    public void setModel(String model) {
        this.model = model;
    }
}

In [5]:
// Owner.java
import java.util.ArrayList;
import java.util.Collections;
import java.util.List;

public class Owner extends Person {
    private final List<Car> cars = new ArrayList<>();

    public Owner(String name) { super(name); }

    public void addCar(Car car) {
        if (car == null) return;
        if (!cars.contains(car)) {
            cars.add(car);
            car.setOwner(this); // bi-directional link
        }
    }

    public void removeCar(Car car) {
        if (car == null) return;
        if (cars.remove(car)) {
            car.setOwner(null); // break link
        }
    }

    public List<Car> getCars() {
        return Collections.unmodifiableList(cars);
    }

    public void setCars(List<Car> cars) {
        for (Car c : new ArrayList<>(this.cars)) c.setOwner(null);
        this.cars.clear();
        if (cars != null) for (Car c : cars) addCar(c);
    }
}

In [6]:
public class Main {
    public static void main(String[] args) {
        Owner owner = new Owner("John Doe");
        Car car1 = new Car("Toyota");
        Car car2 = new Car("Honda");

        // Add cars to the owner
        owner.addCar(car1); // Updates car1 object. Sets car1.owner to "this"
        owner.addCar(car2); // Updates car2 object. Sets car2.owner to "this"

        // Access the cars of the owner
        System.out.println(owner.getName() + " owns the following cars:");
        for (Car car : owner.getCars()) {
            System.out.println(car.getModel());
        }

        // Access the owner of a car
        System.out.println(
            car1.getModel() + " is owned by " +
            (car1.getOwner() != null ? car1.getOwner().getName() : "no one")
        );

        // Remove a car from the owner
        owner.removeCar(car1);
        System.out.println("After removing the Toyota, " + owner.getName() + " owns:");
        for (Car car : owner.getCars()) {
            System.out.println(car.getModel());
        }
    }
}

In [7]:
Main.main(null);

John Doe owns the following cars:
Toyota
Honda
Toyota is owned by John Doe
After removing the Toyota, John Doe owns:
Honda


# Inheritance: Liskov Substitution Principle

In [1]:
interface Shape {
    int area();
}

final class Rectangle implements Shape {
    private final int width, height;
    Rectangle(int width, int height) { this.width = width; this.height = height; }
    @Override public int area() { return width * height; }
}

final class Square implements Shape {
    private final int side;
    Square(int side) { this.side = side; }
    @Override public int area() { return side * side; }
}

class Client {
    static int printDoubleArea(Shape s) {
        int a = s.area();           // Works for any Shape
        System.out.println("area = " + a);
        return 2 * a;
    }

    public static void main(String[] args) {
        printDoubleArea(new Rectangle(5, 4)); // ok
        printDoubleArea(new Square(5));       // ok
    }
}

In [2]:
Client.main(null);

area = 20
area = 25


# Why this satisfies LSP:
The client only assumes “a shape has an area().” Both Rectangle and Square fulfill that contract without adding hidden constraints or changing expected behavior.

In [4]:
class RectangleBad {
    protected int width, height;
    public void setWidth(int w)  { this.width = w; }
    public void setHeight(int h) { this.height = h; }
    public int area()            { return width * height; }
}

class SquareBad extends RectangleBad {
    @Override public void setWidth(int w)  { this.width = this.height = w; }
    @Override public void setHeight(int h) { this.width = this.height = h; }
}

class ClientBad {
    // Client expects width and height to be independent for any RectangleBad
    static int resizeTo(RectangleBad r) {
        r.setWidth(5);
        r.setHeight(4);
        return r.area();     // expects 20
    }

    public static void main(String[] args) {
        System.out.println(resizeTo(new RectangleBad())); // 20 ✅
        System.out.println(resizeTo(new SquareBad()));    // 16 ❌ expectation broken
    }
}

In [5]:
ClientBad.main(null);

20
16


# Why this violates LSP:
The client’s valid assumption for the base type—“setWidth and setHeight are independent”—isn’t true for the subtype SquareBad. Substituting SquareBad changes observable behavior (area becomes 16 instead of 20), so LSP is broken.

# Question: Does method overloading breaks LSB?

In [15]:
class Base {
    void process(String data) { System.out.println(data);}
}

class Sub extends Base {
    void process(String data, int count) { }
}

In [17]:
Base b = new Sub();
b.process("hello");  

hello


Does this break LSP?
	•	Not automatically. Since Java doesn’t allow “changing” the inherited method’s parameters, the base contract is still valid: the subclass must still provide the original method. However, Check the following example

In [11]:
class Base {
    void save(String data) {
        System.out.println("Saving: " + data);
    }
}

class Sub extends Base {
    // override but add hidden requirements
    @Override
    void save(String data) {
        if (data == null) throw new IllegalArgumentException("Data must not be null"); 
        // stricter precondition than Base (bad)
        System.out.println("Saving safely: " + data);
    }
}

In [13]:
Base b = new Sub();
b.save("hello");  

Saving: hello


Here, Sub is stricter than Base—LSP is broken.

## Rule of thumb
	•	Changing method signature → overloading, not overriding → LSP is safe (as long as original behavior is untouched).
	•	Changing behavior of the original method (weaken postconditions or strengthen preconditions) → can break LSP.


# Static Attributes and Variables

In Java (and most OOP languages):
- A static attribute (also called a class variable) belongs to the class itself, not to individual instances (objects).
- This means:
- There is only one copy of the static attribute, shared by all objects of that class.
- Changing it through one object will affect all other objects of that class, because they all refer to the same memory location.
- It can be accessed without creating an object, using the class name.

In [32]:
class Counter {
    static int count = 0;   // static attribute

    public Counter() {
        if(count > 0){return;}
        count++; // every time a new object is created, count increases
    }
}



In [33]:
new Counter();
new Counter();
new Counter();

System.out.println(Counter.count); // prints 3

1


 ## Static Methods
- A static method belongs to the class itself, not to any particular instance.
- You can call it without creating an object, using the class name.
- It cannot access instance variables or instance methods directly, because it doesn’t know which object’s state to use (it has no this reference).

In [39]:
class MathUtils {
    // static method
    public static int add(int a, int b) {
        return a + b;
    }

    // instance method
    public int multiply(int a, int b) {
        return a * b;
    }
}



In [40]:

System.out.println(MathUtils.add(5, 3)); // ✅ prints 8

        // Call instance method (needs object)
MathUtils utils = new MathUtils();
System.out.println(utils.multiply(5, 3)); // ✅ prints 15

8
15


## One example of standard Java

In [42]:
Math.sqrt(25);

5.0